In [14]:
import carla
import random
from multiprocessing import Process
import queue
import re
import time

### Utils functions

In [2]:
SENSOR_TICK = 3

def get_cam_blueprint(world, vehicle_id):
    cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    cam_bp.set_attribute("image_size_x",str(400 + vehicle_id))
    cam_bp.set_attribute("image_size_y",str(300))
    cam_bp.set_attribute("fov",str(100))
    cam_bp.set_attribute("sensor_tick",str(SENSOR_TICK))
    return cam_bp


def get_lidar_blueprint(world, vehicle_id):
    lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
    lidar_bp.set_attribute('sensor_tick', str(SENSOR_TICK))
    #lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('channels', str(vehicle_id))
    lidar_bp.set_attribute('points_per_second', '1120000')
    lidar_bp.set_attribute('upper_fov', '30')
    lidar_bp.set_attribute('range', '100')
    lidar_bp.set_attribute('rotation_frequency', '20')
    return lidar_bp

def geolocation_blueprint(world, vehicle_id):
    gnss_bp = world.get_blueprint_library().find('sensor.other.gnss')
    gnss_bp.set_attribute('sensor_tick', str(SENSOR_TICK))
    return gnss_bp


def retrieve_data(sensor_queue, timeout=5):
    while True:
        try:
            data = sensor_queue.get(True,timeout)
        except queue.Empty:
            return None
        return data

### Creating client

In [3]:
# Connect to the client and retrieve the world object
#client = carla.Client('localhost', 2000)
client = carla.Client('192.168.0.182', 2000)

world = client.get_world()

In [10]:
#world.__dir__()
world.get_map().name

'Carla/Maps/Town10HD_Opt'

In [11]:
# load town 04
# client.load_world('Town04')
# client.load_world('Town01')
# client.load_world('Town03')
# client.load_world('Town02')
client.load_world("Town10HD_Opt")

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to 192.168.0.182:2000

In [18]:
def find_weather_presets():
    rgx = re.compile('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)')
    name = lambda x: ' '.join(m.group(0) for m in rgx.finditer(x))
    presets = [x for x in dir(carla.WeatherParameters) if re.match('[A-Z].+', x)]
    return [(getattr(carla.WeatherParameters, x), name(x)) for x in presets]

presets = find_weather_presets()
presets

[(<carla.libcarla.WeatherParameters at 0x7f95d473eb70>, 'Clear Night'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e390>, 'Clear Noon'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e780>, 'Clear Sunset'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473ec00>, 'Cloudy Night'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e420>, 'Cloudy Noon'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e810>, 'Cloudy Sunset'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e300>, 'Default'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473ef60>, 'Dust Storm'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473eed0>, 'Hard Rain Night'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e660>, 'Hard Rain Noon'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473ea50>, 'Hard Rain Sunset'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473e9c0>, 'Mid Rain Sunset'),
 (<carla.libcarla.WeatherParameters at 0x7f95d473ee40>, 'Mid Rainy Night'),
 (<carla.libcarla.WeatherParameters at 0x7f95d

In [21]:
presets[9]

(<carla.libcarla.WeatherParameters at 0x7f95d473e660>, 'Hard Rain Noon')

In [26]:
presets[9][0]

In [27]:
world.set_weather(presets[9][0])

In [28]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# for blueprint in vehicle_blueprints:
#     print(blueprint.id)

In [29]:
# vehicle.mini.cooper_s
list(vehicle_blueprints)[-1]

### Add vehicles to world

In [34]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

vehicle_num = 10

# add vehicle_num number of vehicles at random spawn_points
for i in range(vehicle_num):
    world.try_spawn_actor(list(vehicle_blueprints)[-1], random.choice(spawn_points))

### Add sensors to cars

In [35]:
camera_init_trans = carla.Transform(carla.Location(z=1.7))
lidar_init_trans = carla.Transform(carla.Location(z=3.0))
spawned_vehicles = world.get_actors().filter('*vehicle*')


In [ ]:
# cam_queue = queue.Queue()
# lidar_queue = queue.Queue()
# cam_list = []
# lidar_list = []
# for vehicle in spawned_vehicles:
#     print(vehicle)

#     # create camera and attach to vehicle
#     camera_bp = get_cam_blueprint(world, vehicle.id)
#     camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
#     #camera.listen(lambda image: image.save_to_disk(f'sensors/vehicle-id-{vehicle.id}-%06d.png' % image.frame))
#     camera.listen(cam_queue.put)
#     cam_list.append(camera)

#     # create lidar sensor and attach to vehicle
#     lidar_bp = get_lidar_blueprint(world, vehicle.id)
#     lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)
#     lidar.listen(lidar_queue.put)
#     lidar_list.append(lidar)

#     # add vehicles to Trafic manager
#     vehicle.set_autopilot(True)
    
#     ## TODO: ADD GNSS for gps and location
#     ## TODO: ADD IMU for speed

### Local part where the data is processed

At this point, data is saved at storage

In the future, this queues will be MQ queues to communicate with OMNET+

In [ ]:
# time_sim = 0
# while True:

#     # sleep before getting data from queues
#     time.sleep(1)
    
#     # get cam data
#     data = retrieve_data(cam_queue)
#     # Skip if any sensor data is not available
#     if None in data:
#         ...
#     else:
#         rgb_img = data
#         # print(rgb_img)
#         rgb_img.save_to_disk(f'sensors/camera/vehicle-{rgb_img.width - 400}-camera-frame-%06d.png' % rgb_img.frame)
    
#     # get lidar data
#     lidar_data = retrieve_data(lidar_queue)
#     if None in data:
#         ...
#     else:
#         #print("lidar data:", lidar_data)
#         lidar_data.save_to_disk(f'sensors/lidar/vehicle-{lidar_data.channels}-frame-%06d.ply' % rgb_img.frame)

In [32]:
for vehicle in spawned_vehicles:
    print(vehicle)

    # add vehicles to Trafic manager
    vehicle.set_autopilot(True)


Actor(id=148, type=vehicle.mini.cooper_s)
Actor(id=147, type=vehicle.mini.cooper_s)
Actor(id=146, type=vehicle.mini.cooper_s)
Actor(id=145, type=vehicle.mini.cooper_s)
Actor(id=144, type=vehicle.mini.cooper_s)
Actor(id=143, type=vehicle.mini.cooper_s)
Actor(id=142, type=vehicle.mini.cooper_s)
Actor(id=141, type=vehicle.mini.cooper_s)
Actor(id=140, type=vehicle.mini.cooper_s)
Actor(id=139, type=vehicle.mini.cooper_s)
Actor(id=138, type=vehicle.mini.cooper_s)
Actor(id=137, type=vehicle.mini.cooper_s)
Actor(id=136, type=vehicle.mini.cooper_s)
Actor(id=135, type=vehicle.mini.cooper_s)
Actor(id=134, type=vehicle.mini.cooper_s)
Actor(id=133, type=vehicle.mini.cooper_s)
Actor(id=132, type=vehicle.mini.cooper_s)
Actor(id=131, type=vehicle.mini.cooper_s)
Actor(id=130, type=vehicle.mini.cooper_s)
Actor(id=129, type=vehicle.mini.cooper_s)
Actor(id=128, type=vehicle.mini.cooper_s)
Actor(id=127, type=vehicle.mini.cooper_s)
Actor(id=126, type=vehicle.mini.cooper_s)
Actor(id=125, type=vehicle.mini.co

In [36]:
for vehicle in spawned_vehicles:
    vehicle.destroy()